# RLWE Homomorphic Encryption Example
This Jupyter Notebook provides an example of using the RLWE homomorphic encryption scheme for encrypting and decrypting binary vectors.

## Import Libraries
We begin by importing the necessary libraries for our example.

In [24]:
import rlwe_he_scheme_updated as rlwe_updated
import numpy as np

## Scheme Parameters
We define the parameters for the RLWE encryption scheme.

$$
\begin{align*}
\text{Polynomial Modulus Degree (n)} &: n = 2^3 = 8 \\
\text{Ciphertext Modulus (q)} &: q = 2^{14} = 16384 \\
\text{Plaintext Modulus (t)} &: t = 2 \\
\text{Base for } \text{relin\_v1 (T)} &: T = \sqrt{q} = \sqrt{16384} = 128 \\
\text{Modulusswitching Modulus (p)} &: p = q^3 = 4398046511104 \\
\text{Polynomial Modulus (poly\_mod)} &: \text{poly\_mod} = [1, \underbrace{0, 0, \ldots, 0}_{n-1}, 1] \\
\text{Standard Deviation (std1)} &: \text{std1} = 1 \\
\text{Standard Deviation (std2)} &: \text{std2} = 1
\end{align*}
$$

These parameters are essential for the RLWE homomorphic encryption scheme and are used for various encryption and evaluation operations.


In [25]:
# Scheme's parameters
# polynomial modulus degree
n = 2 ** 3
# ciphertext modulus
q = 2 ** 14
# plaintext modulus
t = 2
# base for relin_v1
T = int(np.sqrt(q))
# modulusswitching modulus
p = q ** 3

# polynomial modulus
poly_mod = np.array([1] + [0] * (n - 1) + [1])

# standard deviation for the error in the encryption
std1 = 1
# standard deviation for the error in the evaluateKeyGen_v2
std2 = 1
n, q, t, T, p, poly_mod, std1, std2

(8, 16384, 2, 128, 4398046511104, array([1, 0, 0, 0, 0, 0, 0, 0, 1]), 1, 1)

## Key Generation
The key generation process involves generating a public key $ pk $ and a secret key $ sk $. The public key is used for encryption, and the secret key is used for decryption.

### Generating Secret and Public Keys

The secret key $ sk $ is a binary polynomial. The public key $ pk $ is generated as follows:

$$
\begin{align*}
sk &: \text{Binary secret key polynomial} \\
a &: \text{Uniformly random polynomial} \\
e &: \text{Error polynomial sampled from a normal distribution} \\
b &: a \cdot sk - e \mod q
\end{align*}
$$

After the key generation process, we obtain the secret key $ sk $ and the public key $ pk $, which are used in various encryption and evaluation operations.


In [26]:
# Keygen
pk, sk = rlwe_updated.keygen(n, q, poly_mod, std1)
pk, sk

((array([15921, 10402,   248, 12723,  5245,  5517, 12179,  6717],
        dtype=int64),
  array([ 1601, 12640, 10112, 16156,  2832, 10579, 11613, 15401],
        dtype=int64)),
 array([0, 0, 1, 1, 0, 0, 1, 0], dtype=int64))

# Encrypting the Binary Vectors

Once we have the binary vectors representing the characters in the message "Hello world," we proceed to encrypt each binary vector to obtain a set of ciphertexts. The encryption process involves converting each binary vector into an encrypted form using the public key $ pk $ and other encryption parameters.

$$
\begin{align*}
\text{ciphertexts} &: \text{List of encrypted binary vectors} \\
\text{binary\_vector} &: [\text{"01001000"}, \text{"01100101"}, \text{"01101100"}, \\
&\quad \text{"01101100"}, \text{"01101111"}, \text{"00100000"}, \text{"01110111"}, \\
&\quad \text{"01101111"}, \text{"01110010"}, \text{"01101100"}, \text{"01100100"}] \\
\text{pt} &: \text{Binary vector converted to a list of integers} \\
\text{ct} &: \text{Encrypted ciphertext obtained from the encryption function}
\end{align*}
$$

The resulting $ \text{ciphertexts} $ represent the encrypted form of the binary vectors corresponding to the characters in the message "Hello world." These ciphertexts preserve the privacy of the original message while allowing for further operations to be performed on the encrypted data.


In [27]:
# Encrypting "Hello world" as binary vectors
plaintext = "Hello world"
# Convert each character to 8-bit binary
binary_vector = [format(ord(char), '08b') for char in plaintext]
plaintext, binary_vector

('Hello world',
 ['01001000',
  '01100101',
  '01101100',
  '01101100',
  '01101111',
  '00100000',
  '01110111',
  '01101111',
  '01110010',
  '01101100',
  '01100100'])

In [28]:
# Encrypting the binary vectors
ciphertexts = []
for binary_value in binary_vector:
    pt = [int(bit) for bit in binary_value]
    ct = rlwe_updated.encrypt(pk, n, q, t, poly_mod, pt, std1)
    ciphertexts.append(ct)
ciphertexts

[(array([13354, 10782,  7754, 16081,  1285,  1804,  4782,  9038],
        dtype=int64),
  array([ 9995,  7330, 14340,  2227, 15765,   638, 14760,  1987],
        dtype=int64)),
 (array([15017,  5047, 14961,  2647,  1908, 15865, 16194,  3579],
        dtype=int64),
  array([13217, 14244,  8955, 10326,  9417, 13723,  8724, 10573],
        dtype=int64)),
 (array([ 9939,  4888,  6146, 16024,  7128,  2020, 14449, 15456],
        dtype=int64),
  array([14787,  7483, 15381, 15613,  4305,  5527,  5416,  8436],
        dtype=int64)),
 (array([  488,  6086, 10812, 15289,  6416, 16082,  9452,  6278],
        dtype=int64),
  array([ 7185, 12305,  2168,  3956, 12178,  2010, 12696, 14014],
        dtype=int64)),
 (array([ 6771, 10840, 10103,  7675,  8001,  3579,  9559, 11336],
        dtype=int64),
  array([ 8955, 10328,  9416, 13722,  8724, 10575,  3167,  2140],
        dtype=int64)),
 (array([ 9283,  9827,   164, 13409,  3221, 10187,  6989,  1833],
        dtype=int64),
  array([ 3200,  7744, 1156

## Decrypting the Binary Vectors and Converting Back to Text

After encrypting the binary vectors representing the characters in the message "Hello world," we proceed to decrypt the ciphertexts and convert the decrypted binary vectors back into text. The decryption process involves using the secret key $ sk $ to reverse the encryption process and obtain the original binary vectors.

$$
\begin{align*}
\text{decrypted\_binary\_vectors} &: \text{List of decrypted binary vectors} \\
\text{ct} &: \text{Encrypted ciphertext to be decrypted} \\
\text{decrypted\_value} &: \text{Decrypted binary vector obtained from the decryption function}
\end{align*}
$$

By decrypting the ciphertexts, we recover the binary vectors that represent the characters in the original message. The resulting $ \text{decrypted\_binary\_vectors} $ are the binary representations of the characters "Hello world" before encryption.


In [6]:
# Decrypting the binary vectors and converting back to text
decrypted_binary_vectors = []
for ct in ciphertexts:
    decrypted_value = rlwe_updated.decrypt(sk, n, q, t, poly_mod, ct)
    decrypted_binary_vectors.append(decrypted_value)
decrypted_binary_vectors

## Converting Decrypted Binary Vectors to Text

To obtain the final decrypted message, we convert the decrypted binary vectors back into their corresponding text representation. This involves reversing the process of converting characters to 8-bit binary vectors and mapping them back to their original characters.

$$
\begin{align*}
\text{decrypted\_text} &: \text{Decrypted message obtained from joining decrypted binary vectors}
\end{align*}
$$

The $ \text{decrypted\_text} $ represents the original message "Hello world," which has been successfully decrypted from its encrypted form.

In [7]:
decrypted_text = ''.join([chr(int(''.join(map(str, binary_vector)), 2)) for binary_vector in decrypted_binary_vectors])
decrypted_text